# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium

In [3]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os

In [4]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [5]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC

In [6]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [7]:
import plotly.express as px
import plotly.graph_objects as go

# Open and Preprocessing Data

In [8]:
model_num = 7
data_num = 2

In [9]:
# Загрузка данных
df = pd.read_csv(Path["dataset"](model_num, data_num))

In [10]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [11]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [12]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [13]:
# Разделение данных временного ряда на тренировочную и тестовую выборки.
def train_test_split_time_series(df, train_size=0.8):
    split_index = int(len(df) * train_size)
    train_df = df.iloc[:split_index].reset_index(drop=True)
    test_df = df.iloc[split_index:].reset_index(drop=True)
    return train_df, test_df

In [14]:
# Выполнение разделения
train_df, test_df = train_test_split_time_series(df, train_size=0.8)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83163, 26)
Testing data shape: (20791, 26)


# Game Rule

- Изменили window_length тепер оно работает и выставили его на 48
- Кроме того, модели подают нормализованную Close, а также RSI теперь без нормализации
- нормализация StandartScaler

In [15]:
window_length = 48
nb_steps = 80000

In [16]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []
        self.seed = 999
        self.set_seed()


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {0:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = None, options=None, reset_hist=False):
        super().reset(seed=self.seed)
        self.set_seed()
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'shares_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        # obs = np.concatenate([
        #     frame.values,
        #     [self.shares_held],
        #     [self.balance],
        #     [self.net_worth]
        # ])
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.prev_token = self.now_token

        terminated = False
        truncated = False

        self.current_step += 1  # Move to the next time step

        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                # self._sell_all_tokens()
                self.now_token -=1
                terminated = True

        if self.prev_net_worth != 0:
            net_worth_change = self.net_worth - self.prev_net_worth
            percent_change = (net_worth_change) / self.prev_net_worth * 100

            if percent_change > 0:
                self.reward += percent_change
            else:
                self.reward -= percent_change * 2  

            if self.net_worth < self.initial_balance * 0.5:
                self.reward += -50  
                self.logger.info("Net worth dropped below 50% of initial balance.")
                terminated = True
        else:
            self.logger.info("prev_net_worth == 0")

        self.prev_net_worth = self.net_worth

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["shares_held"].append(self.shares_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _sell_all_tokens(self):
        """
        Sell all tokens held at the current price.
        """
        self.current_price = self.df.loc[self.current_step - 1, 'Close_orig']
        shares_to_sell = self.shares_held
        if shares_to_sell > 0:
            # Determine total sale amount
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held = 0
            self.total_shares_sold += shares_to_sell  # Corrected: use shares_to_sell
            self.total_sales_value += total_sale

            # Update net worth
            self.net_worth = self.balance

            self.logger.info(f"Sold all shares of token {self.prev_token} at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("No shares to sell.")


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.shares_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            # Hold
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.shares_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.shares_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.shares_held:
            shares_to_sell = self.shares_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("No shares to sell.")


    def set_seed(self):
        # self.np_random, seed = seeding.np_random(seed)
        np.random.seed(self.seed)
        random.seed(self.seed)
        return [self.seed]


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [17]:
train_env = TradingEnv(train_df)

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}


In [18]:
test_env = TradingEnv(test_df)

{0: 0, 21: 117, 22: 4210, 23: 8429, 24: 12817, 25: 16419}


In [19]:
# check_env(train_env)

# Train Model

## Settings

In [20]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/"  # Directory for TensorBoard logs
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [21]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [22]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        # Log every 1000 steps
    # callback=eval_callback    # Pass the evaluation callback here
)

Logging to ./sac_tensorboard/SAC_31


In [23]:
# Save the trained model
model.save(Path["model"](model_num, data_num))

In [24]:
train_hist = train_env.hist
print(len(train_hist["action"]))
train_hist_df = pd.DataFrame(train_hist)

80000


In [25]:
unique_values = train_hist_df['token'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

# Visualisaton

## Functions

In [26]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [27]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [28]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()

In [29]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [30]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

## Plots

In [31]:
token = 19
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [32]:
train_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,625.751394,999.626125,6536043,0,0.000057,0.074775,0.374249,0,0.000000
1,50,491.621926,1008.224283,8825077,0,0.000059,0.860137,0.214350,0,0.000000
2,51,37.352171,1005.776000,16607625,0,0.000058,0.485662,0.924023,0,0.000000
3,52,22.156797,985.931315,16873394,0,0.000057,3.946144,0.406814,0,0.000000
4,53,16.116126,974.603233,16980302,0,0.000056,2.297945,0.272634,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
79995,29884,1998.865947,2000.005795,1,7,1.139848,-4.999946,-0.973919,22865,41179.857326
79996,29885,1998.865947,2000.019968,1,7,1.154021,-4.999291,-0.960387,22865,41179.857326
79997,29886,1998.865947,2000.010858,1,7,1.144911,-4.999089,-0.945601,22865,41179.857326
79998,29887,1998.865947,2000.023348,1,7,1.157401,-4.999376,-0.961929,22865,41179.857326


In [33]:
train_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,80000.000000,8.000000e+04,8.000000e+04
mean,34095.609438,6.577987e+02,2457.847892,1.430275e+08,8.158375,7.967639e+00,-0.431272,0.247368,6.490350e+09,1.856795e+05
std,24306.527597,3.746938e+03,10923.761924,6.405880e+08,6.239614,1.878149e+01,3.888742,0.672312,2.335243e+10,3.780044e+05
min,49.000000,1.091713e-08,444.259235,0.000000e+00,0.000000,5.400000e-08,-48.700324,-1.000000,0.000000e+00,0.000000e+00
25%,12625.000000,5.639463e-01,821.651136,5.000000e+01,3.000000,1.316800e-04,-3.860853,-0.353300,1.144900e+04,2.588145e+04
50%,30093.500000,9.056115e+01,1046.387611,2.611000e+03,7.000000,6.219718e-02,0.122912,0.457091,7.732755e+05,7.898927e+04
75%,55860.250000,6.972992e+02,1619.333838,7.603412e+06,14.000000,2.643798e+00,1.073076,0.872459,1.790650e+09,1.765649e+05
max,80468.000000,1.604809e+05,211740.978781,9.185566e+09,20.000000,1.065761e+02,225.607514,0.999992,2.036320e+11,4.584097e+06


# Test Model

## Test

In [34]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist

In [35]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20791


In [36]:
test_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,1000.000000,1000.000000,0,20,8.209560,-5.0,-0.691555,0,0.000000
1,50,1000.000000,1000.000000,0,20,8.232353,-5.0,-0.831060,0,0.000000
2,51,1000.000000,1000.000000,0,20,8.255531,-5.0,-0.881441,0,0.000000
3,52,1000.000000,1000.000000,0,20,8.238088,-5.0,-0.872148,0,0.000000
4,53,1000.000000,1000.000000,0,20,8.295958,-5.0,-0.934814,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
20786,2960,577.147183,577.147197,1,21,0.000013,-5.0,-0.924621,1502785646,28883.945467
20787,2961,577.147183,577.147197,1,21,0.000013,-5.0,-0.870088,1502785646,28883.945467
20788,2962,577.147183,577.147197,1,21,0.000013,-5.0,-0.910133,1502785646,28883.945467
20789,2963,577.147183,577.147197,1,21,0.000013,-5.0,-0.974242,1502785646,28883.945467


In [37]:
unique_values = test_hist_df['token'].unique()
unique_values

array([20, 21, 22, 23, 24, 25], dtype=int64)

## Plots

In [56]:
token = 25

In [57]:
plot_token_data(df = test_hist_df, token = token)

In [58]:
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [41]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - full_hist: Dictionary containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

In [42]:
# Example for one model
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

In [43]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,20,5.046139,0.504614,0.549394,-1.034865,11.594203,-118.979657,-11.897966,230.293410,23.029341,-11.897966
1,21,-457.376778,-45.737678,-0.147658,-59.349139,27.863822,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
2,22,50346.418961,5034.641896,0.746994,-70.199079,48.141932,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
3,23,-169.298822,-16.929882,0.167074,-83.530074,48.986175,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
4,24,-521.037457,-52.103746,-0.018391,-65.364599,50.551121,594.470161,59.447016,2103.489180,210.348918,59.447016
5,25,-522.826404,-52.282640,-0.372620,-52.019851,49.343832,233.832793,23.383279,2790.670479,279.067048,23.383279


In [44]:
test_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,20791.000000,2.079100e+04,20791.000000,2.079100e+04,20791.000000,20791.000000,20791.000000,20791.000000,2.079100e+04,20791.000000
mean,8210.225626,3.087381e+02,1612.078565,9.959361e+06,22.504209,46.903609,-2.478532,0.351278,2.906872e+08,18588.769211
std,5728.083074,9.798757e+02,3704.367082,1.793635e+07,1.403928,121.461823,3.221335,0.733999,5.135000e+08,17331.583907
min,49.000000,4.691078e-07,477.173596,0.000000e+00,20.000000,0.000011,-44.828455,-0.999991,0.000000e+00,0.000000
25%,2695.500000,6.688024e-03,754.165059,2.000000e+00,21.000000,0.000024,-4.829749,-0.306281,3.623000e+03,3787.644189
50%,7672.000000,5.225439e+00,984.304525,2.912000e+03,22.000000,0.031095,-3.862133,0.753557,1.003692e+06,16097.868617
75%,12965.500000,5.730779e+02,1227.577010,9.630410e+05,24.000000,4.825309,0.004040,0.966204,4.730777e+08,26580.457427
max,19134.000000,2.459441e+04,59325.448639,4.607740e+07,25.000000,588.891708,48.205306,1.000000,1.945394e+09,121552.418750


In [45]:
report_df.to_csv(Path["reports"](model_num, data_num))

In [46]:
overall_metrics

{'Total Profit': 48680.925638450535,
 'ROI (%)': 811.3487606408423,
 'Sharpe Ratio': 0.1541321236828915,
 'Win Rate (%)': 39.41351433297815,
 'Buy-and-Hold Profit': 254079.44959758603,
 'Buy-and-Hold ROI (%)': 4234.657493293101}